In [12]:
import pandas as pd
from datetime import date
import re
import csv

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [3]:
class Formatar():
    def __init__(self, df, colunas_data):
        self.df = df
        self.colunas_data = colunas_data
        self.coluna_metadados = 'metadados'

    def info_adicional(self, texto):
        info_adicional = [ 
        'ENEM'
        'EAD'
        'SISU'
        'Cancelado'
        ]
        texto_final = ''
        for info in info_adicional:
            if info in texto:
                texto_final += f'''{info}\n''' 
        return f'''{self.coluna}: {texto_final}''' if texto_final != '' else ''

    def encontrar_datas(self, texto):
        r = re.compile(r'\d{2}/\d{2}/\d{4}')
        s =  re.compile(r'(\d{2}/\d{2}[^/0-9])')
        r = str(r.findall(texto))[2:-2]
        s = str(s.findall(texto))[2:-3]
        if r != '':
            self.dia = r[:2]
            self.mes = r[3:5]
            self.ano = r[6:]
            self.data = f'''{self.ano}-{self.mes}-{self.dia}'''
        elif s != '':
            self.dia = s[:2]
            self.mes = s[3:5]
            self.data = f'''-{self.mes}-{self.dia}'''
        else:
            self.data = ''
            self.metadados = self.info_adicional(texto)
    
    def adicionar_ano(self, texto):
        r = re.compile(r'\d{2}/\d{2}/\d{4}')
        r = str(r.findall(texto))[2:-2]
        if r != '':
            self.proximo_dia = r[:2]
            self.proximo_mes = r[3:5]
            self.proximo_ano = r[6:]
        else:
            self.proximo_dia = ''
            self.proximo_mes = ''
            self.proximo_ano = ''

    def tratar_datas(self):
        for index, row in self.df.iterrows():
            for i in range(len(self.colunas_data)):
                self.coluna = self.colunas_data[i] 
                try:
                    self.proxima_coluna = self.colunas_data[i+1] #caso a data não tenha o ano, pegamos da próxima data (ex: incrições de 20/10 a 20/11/2021)
                    self.adicionar_ano(row[self.proxima_coluna])
                except:
                    self.proxima_coluna = ''
                self.metadados = ''
                self.encontrar_datas(row[self.coluna])
                if self.data != '':
                    if str(self.data)[0] == '-':
                        self.data = str(self.proximo_ano) + str(self.data)
                self.df.loc[index, self.coluna_metadados] += self.metadados
                self.df.loc[index, self.coluna] = self.data

            

In [4]:
df = pd.read_csv('ranking.csv', sep = ',', encoding = 'utf-8', dtype = str)
df.fillna('', inplace = True)

colunas = [
    'Inicio incrição', 'Fim inscrição', 'Início isenção ', 'Fim isenção',
    'Primiera fase', 'Primeira fase (segundo dia)', 'Segunda fase',
    'Segunda fase (segundo dia)', 'Resultado'
    ]



In [5]:
df_teste = Formatar(df, colunas)

In [6]:
df_teste.tratar_datas()

In [11]:
df_teste.df.head()

,Ranking,Instituição,Pública ou Privada,Estado,Inicio incrição,Fim inscrição,Início isenção,Fim isenção,Primiera fase,Primeira fase (segundo dia),Segunda fase,Segunda fase (segundo dia),Resultado,link site,metadados,Posição em Ensino,Nota em Ensino,Posição em Pesquisa,Nota em Pesquisa,Posição em Mercado,Nota em Mercado,Posição em Inovação,Nota em Inovação,Posição em Internacionalização,Nota em Internacionalização,Nota,Extrator
0,1,USP,,SP,2021-08-16,2021-10-01,2021-06-14,2021-08-07,2021-12-12,,2022-01-16,,,https://www.fuvest.br/wp-content/uploads/fuves...,,,,,,,,,,,,,
1,2,UNICAMP,,SP,2021-09-02,2021-09-08,2021-07-30,2021-08-02,2021-11-07,,2022-01-09,,,https://www.comvest.unicamp.br/publicado-o-edi...,,,,,,,,,,,,,
2,3,UFRJ,,RJ,,,,,,,,,,,,,,,,,,,,,,,
3,4,UFMG,,MG,,,,,,,,,,,,,,,,,,,,,,,
4,5,UFRGS,,RS,,,,,,,,,,,,,,,,,,,,,,,


In [8]:
def order_ranking(df):
    df['Ranking'] = df['Ranking'].astype(int)
    df = df.sort_values(by = 'Ranking')
    return display(df)

In [14]:
df_teste.df.to_csv('ranking_parcial(formatado).csv', sep = ',', encoding = 'utf-8', index = False, quoting = csv.QUOTE_ALL)